# Import Modules and Data

In [1]:
# Load modules
import sys
import os
os.chdir('..')

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from functions import cvmodeleval,samplecv, trainsampling

In [2]:
# Load training filename dataframe
df = pd.read_csv('data/processed/driver_image_list_processed.csv')

In [3]:
# Labeled and unlabeled test filename dataframes
df_test_labeled = pd.read_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/processed/labeled_test_df.csv')
df_test = pd.read_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/raw/test_filenames.csv')

In [4]:
# Call custom function to over/undersample classes occurance by subject so dataset is completely balanced.
df = trainsampling(df, samples=80, random_state=42)

In [5]:
# Define path variables for data
train_path = 'D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/raw/imgs/train'
labeled_test_path = 'D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/raw/imgs/testlabeled'
unlabeled_test_path = 'D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/raw/imgs/test'

4 subjects were chosen from the training data to be used for validation during model training. These subjects represent one woman and man with dark skin and one woman and man with light skin. This is to help balance any potential racial bias in the model.

In [6]:
# Define list of validation subjects
val_subjects = ['p056', 'p050', 'p041', 'p016']

In [7]:
# Define training and validation datasets
df_train = df[~df['subject'].isin(val_subjects)]
df_val = df[df['subject'].isin(val_subjects)]

In [8]:
# Shuffle the dataframes
df_train = df_train.sample(frac=1, random_state=42)
df_val = df_val.sample(frac=1, random_state=42)

## Pre-Processing and Data-Loading

In [11]:
# Define ImageDataGenerators for training and test/validation data. Generators include randomized preprocessing for 
# called out parameters. Test_dgen will be used for both validation data and test data.

train_dgen = ImageDataGenerator(samplewise_center=True,
                                rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                channel_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                brightness_range=[0.5, 1.5])

test_dgen = ImageDataGenerator(samplewise_center=True,
                               rescale=1./255)

In [12]:
# Load training, validation, and test data
train = train_dgen.flow_from_dataframe(df_train,
                                       x_col='imgpath',
                                       y_col='classname',
                                       batch_size=16, 
                                       target_size=(227,227),
                                       shuffle=True)

val = test_dgen.flow_from_dataframe(df_val,
                                    x_col='imgpath',
                                    y_col='classname',
                                    target_size=(227,227),
                                    shuffle=False)

test_labeled = test_dgen.flow_from_dataframe(df_test_labeled,
                                             x_col='filename',
                                             y_col='classname',
                                             target_size=(227,227),
                                             shuffle=False)

test = test_dgen.flow_from_dataframe(df_test,
                                     x_col='filename',
                                     y_col='class',
                                     target_size=(227,227),
                                     shuffle=False)


Found 17600 validated image filenames belonging to 10 classes.
Found 3200 validated image filenames belonging to 10 classes.
Found 200 validated image filenames belonging to 10 classes.
Found 79726 validated image filenames belonging to 1 classes.


# Model1 - AlexNet Architecture

Architecture below is based off of AlexNet

In [12]:
# Define function for initializing model
def alexNet_arch(opt):
    model = Sequential()
    model.add(Conv2D(99,
                  kernel_size=11,
                  strides=4,
                  padding='valid',
                  input_shape=(227, 227, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(3,
                         strides=2,
                         padding='valid'))
    model.add(Conv2D(256,
                      kernel_size=5,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(3,
                        strides=2,
                        padding='valid'))
    model.add(Conv2D(384,
                      kernel_size=3,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(384,
                      kernel_size=3,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256,
                      kernel_size=3,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(3,
                         strides=2,
                         padding='valid'))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
# Define optimizer variable
opt = RMSprop(learning_rate=0.0001)

In [14]:
# Compile model_1
model_1 = alexNet_arch(opt)

In [15]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model1_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=20, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [16]:
%%time
# Train model_1
model_1.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

Epoch 1/100
110/110 [==============================] - 37s 323ms/step - loss: 6.9370 - accuracy: 0.0918 - val_loss: 2.4704 - val_accuracy: 0.1000
Epoch 2/100
110/110 [==============================] - 36s 326ms/step - loss: 2.7232 - accuracy: 0.0996 - val_loss: 2.3143 - val_accuracy: 0.1000
Epoch 3/100
110/110 [==============================] - 35s 320ms/step - loss: 2.4712 - accuracy: 0.1074 - val_loss: 2.3308 - val_accuracy: 0.1000
Epoch 4/100
110/110 [==============================] - 35s 319ms/step - loss: 2.4106 - accuracy: 0.1175 - val_loss: 2.3382 - val_accuracy: 0.0819
Epoch 5/100
110/110 [==============================] - 34s 312ms/step - loss: 2.3365 - accuracy: 0.1322 - val_loss: 2.2670 - val_accuracy: 0.1278
Epoch 6/100
110/110 [==============================] - 34s 311ms/step - loss: 2.2182 - accuracy: 0.1680 - val_loss: 2.1761 - val_accuracy: 0.1900
Epoch 7/100
110/110 [==============================] - 34s 311ms/step - loss: 2.1847 - accuracy: 0.1627 - val_loss: 2.0818 -

In [17]:
# Save model predictions of validation data
validation_predictions = model_1.predict(val)

In [18]:
# Convert to a dataframe with original labeles
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
validation_predictions = df_val.reset_index(drop=True).join(validation_predictions)

In [19]:
# Save validation predictions
validation_predictions.to_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/validation_predictions/model_1_validation_predictions.csv')

In [20]:
# Evaluate validation data with best coefficients
model_1_val_metrics = model_1.evaluate(val)

100/100 [==============================] - 10s 105ms/step - loss: 0.9719 - accuracy: 0.6928


In [21]:
model_1_val_metrics = pd.DataFrame({'Cross Entropy Loss':[model_1_val_metrics[0]], 
                                    'Accuracy':[model_1_val_metrics[1]]})
model_1_val_metrics.to_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/metrics/model_1_val_metrics.csv', index=False)

In [22]:
model_1_val_metrics

,Cross Entropy Loss,Accuracy
0,0.971922,0.692813


0.9719 cross-entropy loss and 0.6928 accuracy are not a bad start

### Model 2 - Xception Transfer Learning

Using a method desribed at this url (https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/), I loaded a pre-trained Xception model through keras with weights optimized for imagenet. I made all of the existing layers non-trainable and then added a few trainable layers which will be fit to the data.

In [12]:
base_model_2 = tf.keras.applications.Xception(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_2.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_2.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_2 = tf.keras.models.Model(base_model_2.input, x)
model_2.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 113, 113, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 113, 113, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 113, 113, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [13]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model2_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=20, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [14]:
model_2.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/100
110/110 [==============================] - 71s 647ms/step - loss: 2.8249 - accuracy: 0.1415 - val_loss: 2.1583 - val_accuracy: 0.2169
Epoch 2/100
110/110 [==============================] - 60s 549ms/step - loss: 2.3187 - accuracy: 0.1727 - val_loss: 2.2208 - val_accuracy: 0.2288
Epoch 3/100
110/110 [==============================] - 62s 560ms/step - loss: 2.1792 - accuracy: 0.2364 - val_loss: 2.4904 - val_accuracy: 0.1937
Epoch 4/100
110/110 [==============================] - 60s 548ms/step - loss: 2.1252 - accuracy: 0.2557 - val_loss: 2.3664 - val_accuracy: 0.2147
Epoch 5/100
110/110 [==============================] - 59s 535ms/step - loss: 2.0460 - accuracy: 0.3051 - val_loss: 2.4786 - val_accuracy: 0.2256
Epoch 6/100
110/110 [==============================] - 58s 526ms/step - loss: 1.9675 - accuracy: 0.3210 - val_loss: 2.5725 - val_accuracy: 0.2037
Epoch 7/100
110/110 [=========

In [15]:
# Save model predictions of validation data
validation_predictions = model_2.predict(val)

# Convert to a dataframe with original labels
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
validation_predictions = df_val.reset_index(drop=True).join(validation_predictions)

In [16]:
# Save validation predictions
validation_predictions.to_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/validation_predictions/model_2_validation_predictions.csv')

In [17]:
# Evaluate validation data with best coefficients
model_2_val_metrics = model_2.evaluate(val)

  ...
    to  
  ['...']
100/100 [==============================] - 19s 192ms/step - loss: 2.1583 - accuracy: 0.2169


In [21]:
model_2_val_metrics = pd.DataFrame({'Cross Entropy Loss':[model_2_val_metrics[0]], 
                                    'Accuracy':[model_2_val_metrics[1]]})
model_2_val_metrics.to_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/metrics/model_2_val_metrics.csv', index=False)

In [22]:
model_2_val_metrics

,Cross Entropy Loss,Accuracy
0,2.15828,0.216875


### Model 3 - VGG16 Transfer-Learning

In [11]:
base_model_3 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_3.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_3.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_3 = tf.keras.models.Model(base_model_3.input, x)
model_3.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model3_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=50, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [14]:
model_3.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/100
110/110 [==============================] - 63s 569ms/step - loss: 2.5059 - accuracy: 0.1324 - val_loss: 2.1846 - val_accuracy: 0.1606
Epoch 2/100
110/110 [==============================] - 59s 541ms/step - loss: 2.2511 - accuracy: 0.1653 - val_loss: 2.0830 - val_accuracy: 0.2734
Epoch 3/100
110/110 [==============================] - 59s 538ms/step - loss: 2.1532 - accuracy: 0.2284 - val_loss: 1.9477 - val_accuracy: 0.2266
Epoch 4/100
110/110 [==============================] - 59s 535ms/step - loss: 2.1002 - accuracy: 0.2295 - val_loss: 1.8449 - val_accuracy: 0.3663
Epoch 5/100
110/110 [==============================] - 59s 537ms/step - loss: 1.9959 - accuracy: 0.2886 - val_loss: 1.6320 - val_accuracy: 0.5341
Epoch 6/100
110/110 [==============================] - 60s 543ms/step - loss: 1.9342 - accuracy: 0.3034 - val_loss: 1.5335 - val_accuracy: 0.6256
Epoch 7/100
110/110 [=========

I renamed the weight file for this first run to model3_weights_first_run.hdf5

In [12]:
# Load weights from first run
model_3.load_weights('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model3_weights_first_run.hdf5')

In [13]:
# Save model predictions of validation data
validation_predictions = model_3.predict(val)

# Convert to a dataframe with original labels
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
validation_predictions = df_val.reset_index(drop=True).join(validation_predictions)

In [16]:
# Save validation predictions
validation_predictions.to_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/validation_predictions/model_3_validation_predictions.csv')

In [14]:
# Evaluate validation data with best coefficients
model_3_val_metrics = model_3.evaluate(val)

100/100 [==============================] - 15s 150ms/step - loss: 0.7258 - accuracy: 0.7607


In [18]:
model_3_val_metrics = pd.DataFrame({'Cross Entropy Loss':[model_3_val_metrics[0]], 
                                    'Accuracy':[model_3_val_metrics[1]]})
model_3_val_metrics.to_csv('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/metrics/model_3_val_metrics.csv', index=False)

In [15]:
model_3_val_metrics

[0.7318361401557922, 0.7568749785423279]

Getting better!

# Ensemble Learning

In order to cross-validate, the same pre-trained VGG16 Model will be trained on 4 additional CV-folds. The predictions from all 4 models will be ensembled together to ideally improve model performance.

In [8]:
# Define validation subjects for upcoming CV-folds.
val_subjects_cv2 = ['p002', 'p014', 'p022', 'p042']
val_subjects_cv3 = ['p012', 'p015', 'p045', 'p049']
val_subjects_cv4 = ['p075', 'p066', 'p064', 'p051']
val_subjects_cv5 = ['p052', 'p047', 'p061', 'p035']

## 2nd CV Fold

In [41]:
# Define training and validation datasets
df_train = df[~df['subject'].isin(val_subjects_cv2)]
df_val = df[df['subject'].isin(val_subjects_cv2)]

# Shuffle the dataframes
df_train = df_train.sample(frac=1, random_state=42)
df_val = df_val.sample(frac=1, random_state=42)

In [42]:
# Load training, validation, and test data
train = train_dgen.flow_from_dataframe(df_train,
                                       x_col='imgpath',
                                       y_col='classname',
                                       batch_size=16, 
                                       target_size=(227,227),
                                       shuffle=True)

val = test_dgen.flow_from_dataframe(df_val,
                                    x_col='imgpath',
                                    y_col='classname',
                                    target_size=(227,227),
                                    shuffle=False)

Found 17600 validated image filenames belonging to 10 classes.
Found 3200 validated image filenames belonging to 10 classes.


In [43]:
base_model_cv2 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_cv2.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_cv2.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_cv2 = tf.keras.models.Model(base_model_cv2.input, x)
model_cv2.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_cv2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [44]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model_cv2_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=50, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [45]:
model_cv2.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

Epoch 1/100
110/110 [==============================] - 41s 339ms/step - loss: 2.7870 - accuracy: 0.1189 - val_loss: 2.1574 - val_accuracy: 0.2506
Epoch 2/100
110/110 [==============================] - 37s 335ms/step - loss: 2.2966 - accuracy: 0.1594 - val_loss: 2.0629 - val_accuracy: 0.3734
Epoch 3/100
110/110 [==============================] - 37s 337ms/step - loss: 2.2214 - accuracy: 0.1985 - val_loss: 1.9174 - val_accuracy: 0.3703
Epoch 4/100
110/110 [==============================] - 37s 337ms/step - loss: 2.1209 - accuracy: 0.2446 - val_loss: 1.7821 - val_accuracy: 0.4206
Epoch 5/100
110/110 [==============================] - 37s 339ms/step - loss: 2.0485 - accuracy: 0.2555 - val_loss: 1.6020 - val_accuracy: 0.5603
Epoch 6/100
110/110 [==============================] - 37s 341ms/step - loss: 1.9052 - accuracy: 0.3346 - val_loss: 1.5451 - val_accuracy: 0.5891
Epoch 7/100
110/110 [==============================] - 37s 339ms/step - loss: 1.8237 - accuracy: 0.3529 - val_loss: 1.4881 -

Epoch 57/100
110/110 [==============================] - 54s 490ms/step - loss: 0.8335 - accuracy: 0.7247 - val_loss: 1.1995 - val_accuracy: 0.6187
Epoch 58/100
110/110 [==============================] - 55s 499ms/step - loss: 0.8526 - accuracy: 0.7134 - val_loss: 1.1082 - val_accuracy: 0.6319
Epoch 59/100
110/110 [==============================] - 54s 492ms/step - loss: 0.8914 - accuracy: 0.7178 - val_loss: 1.0534 - val_accuracy: 0.6459
Epoch 60/100
110/110 [==============================] - 54s 489ms/step - loss: 0.8394 - accuracy: 0.7077 - val_loss: 1.1749 - val_accuracy: 0.6094
Epoch 61/100
110/110 [==============================] - 55s 501ms/step - loss: 0.8446 - accuracy: 0.7246 - val_loss: 1.3130 - val_accuracy: 0.6087
Epoch 62/100
110/110 [==============================] - 55s 499ms/step - loss: 0.8154 - accuracy: 0.7322 - val_loss: 1.1643 - val_accuracy: 0.6263
Epoch 63/100
110/110 [==============================] - 55s 501ms/step - loss: 0.8065 - accuracy: 0.7292 - val_loss: 1

## 3rd CV Fold

In [52]:
tf.keras.backend.clear_session()

In [9]:
# Define training and validation datasets
df_train = df[~df['subject'].isin(val_subjects_cv3)]
df_val = df[df['subject'].isin(val_subjects_cv3)]

# Shuffle the dataframes
df_train = df_train.sample(frac=1, random_state=42)
df_val = df_val.sample(frac=1, random_state=42)

In [13]:
# Load training, validation, and test data
train = train_dgen.flow_from_dataframe(df_train,
                                       x_col='imgpath',
                                       y_col='classname',
                                       batch_size=16, 
                                       target_size=(227,227),
                                       shuffle=True)

val = test_dgen.flow_from_dataframe(df_val,
                                    x_col='imgpath',
                                    y_col='classname',
                                    target_size=(227,227),
                                    shuffle=False)

Found 17600 validated image filenames belonging to 10 classes.
Found 3200 validated image filenames belonging to 10 classes.


In [14]:
base_model_cv3 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_cv3.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_cv3.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_cv3 = tf.keras.models.Model(base_model_cv3.input, x)
model_cv3.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_cv3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model_cv3_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=50, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [ ]:
model_cv3.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

Epoch 1/100
110/110 [==============================] - 47s 392ms/step - loss: 2.7424 - accuracy: 0.1260 - val_loss: 2.1902 - val_accuracy: 0.2503
Epoch 2/100
110/110 [==============================] - 44s 403ms/step - loss: 2.2875 - accuracy: 0.1674 - val_loss: 2.0469 - val_accuracy: 0.2606
Epoch 3/100
110/110 [==============================] - 39s 359ms/step - loss: 2.1808 - accuracy: 0.2044 - val_loss: 1.9034 - val_accuracy: 0.2828
Epoch 4/100
110/110 [==============================] - 39s 351ms/step - loss: 2.0851 - accuracy: 0.2625 - val_loss: 1.8203 - val_accuracy: 0.2703
Epoch 5/100
110/110 [==============================] - 37s 334ms/step - loss: 2.0092 - accuracy: 0.2835 - val_loss: 1.5902 - val_accuracy: 0.6275
Epoch 6/100
110/110 [==============================] - 41s 369ms/step - loss: 1.8904 - accuracy: 0.3277 - val_loss: 1.5494 - val_accuracy: 0.5044
Epoch 7/100
110/110 [==============================] - 37s 333ms/step - loss: 1.8185 - accuracy: 0.3682 - val_loss: 1.4213 -

Epoch 57/100
110/110 [==============================] - 49s 445ms/step - loss: 0.7982 - accuracy: 0.7274 - val_loss: 0.6788 - val_accuracy: 0.7487
Epoch 58/100
110/110 [==============================] - 51s 468ms/step - loss: 0.8453 - accuracy: 0.7117 - val_loss: 0.7735 - val_accuracy: 0.7166
Epoch 59/100
110/110 [==============================] - 52s 474ms/step - loss: 0.8337 - accuracy: 0.7278 - val_loss: 0.7113 - val_accuracy: 0.7397
Epoch 60/100
110/110 [==============================] - 50s 458ms/step - loss: 0.8714 - accuracy: 0.7237 - val_loss: 0.7467 - val_accuracy: 0.7425
Epoch 61/100
110/110 [==============================] - 52s 474ms/step - loss: 0.8897 - accuracy: 0.6924 - val_loss: 0.6816 - val_accuracy: 0.7694
Epoch 62/100
110/110 [==============================] - 51s 467ms/step - loss: 0.8193 - accuracy: 0.7173 - val_loss: 0.9177 - val_accuracy: 0.6747
Epoch 63/100
110/110 [==============================] - 52s 469ms/step - loss: 0.7908 - accuracy: 0.7470 - val_loss: 0

## 4th CV Fold

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Define training and validation datasets
df_train = df[~df['subject'].isin(val_subjects_cv4)]
df_val = df[df['subject'].isin(val_subjects_cv4)]

# Shuffle the dataframes
df_train = df_train.sample(frac=1, random_state=42)
df_val = df_val.sample(frac=1, random_state=42)

In [ ]:
# Load training, validation, and test data
train = train_dgen.flow_from_dataframe(df_train,
                                       x_col='imgpath',
                                       y_col='classname',
                                       batch_size=16, 
                                       target_size=(227,227),
                                       shuffle=True)

val = test_dgen.flow_from_dataframe(df_val,
                                    x_col='imgpath',
                                    y_col='classname',
                                    target_size=(227,227),
                                    shuffle=False)

In [ ]:
base_model_cv4 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_cv4.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_cv4.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_cv4 = tf.keras.models.Model(base_model_cv4.input, x)
model_cv4.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_cv4.summary()

In [ ]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model_cv4_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=50, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [ ]:
model_cv4.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

## 5th CV Fold

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Define training and validation datasets
df_train = df[~df['subject'].isin(val_subjects_cv5)]
df_val = df[df['subject'].isin(val_subjects_cv5)]

# Shuffle the dataframes
df_train = df_train.sample(frac=1, random_state=42)
df_val = df_val.sample(frac=1, random_state=42)

In [ ]:
# Load training, validation, and test data
train = train_dgen.flow_from_dataframe(df_train,
                                       x_col='imgpath',
                                       y_col='classname',
                                       batch_size=16, 
                                       target_size=(227,227),
                                       shuffle=True)

val = test_dgen.flow_from_dataframe(df_val,
                                    x_col='imgpath',
                                    y_col='classname',
                                    target_size=(227,227),
                                    shuffle=False)

In [ ]:
base_model_cv5 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_cv5.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_cv5.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_cv5 = tf.keras.models.Model(base_model_cv5.input, x)
model_cv5.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_cv5.summary()

In [ ]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('D:/Users/Dylan/Documents/Data Science/Projects/DistractedDrivers/data/weights/model_cv5_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=50, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [ ]:
model_cv5.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

In [17]:
test_nn_output = model_3.predict(test)

In [34]:
test_predictions_df = pd.DataFrame(test_nn_output, 
                              columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [35]:
test_filenames = os.listdir('D:\\Users\\Dylan\\Documents\\Data Science\\Projects\\DistractedDrivers\\data\\raw\\imgs\\test\\test')
test_filenames = pd.DataFrame({'filename':test_filenames})

In [37]:
test_predictions_df = test_filenames.join(test_predictions)
test_predictions_df.rename(columns={'filename':'img'}, inplace=True)
test_predictions_df

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.001619,0.004821,0.011293,0.027774,0.004447,0.851118,0.080587,2.725571e-03,0.011653,0.003962
1,img_10.jpg,0.009783,0.000062,0.000048,0.001064,0.000109,0.988080,0.000144,4.734514e-05,0.000493,0.000171
2,img_100.jpg,0.442520,0.191491,0.027255,0.000582,0.000186,0.003014,0.009266,8.597373e-02,0.012470,0.227242
3,img_1000.jpg,0.000202,0.000092,0.032745,0.000010,0.000036,0.000366,0.000595,4.882125e-05,0.964899,0.001007
4,img_100000.jpg,0.013909,0.000152,0.000098,0.912742,0.070103,0.000915,0.000669,1.139908e-05,0.001219,0.000180
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,0.000934,0.038359,0.027704,0.000053,0.000013,0.004159,0.006234,8.648908e-01,0.022365,0.035290
79722,img_99995.jpg,0.020845,0.015850,0.000383,0.960031,0.000601,0.000080,0.000600,8.723551e-06,0.000224,0.001378
79723,img_99996.jpg,0.000387,0.019609,0.000071,0.955001,0.022456,0.000004,0.002145,9.573871e-07,0.000252,0.000074
79724,img_99998.jpg,0.000054,0.005063,0.303719,0.000002,0.000006,0.000013,0.448532,2.306005e-01,0.011783,0.000227


In [39]:
test_predictions_df.to_csv('D:\\Users\\Dylan\\Documents\\Data Science\\Projects\\DistractedDrivers\\data\\test_predictions(VGG16_Transfer_Learning_2).csv', index=False)